# Image Preprocessing

O pré-processamento de imagens é uma etapa crucial na preparação de imagens para aprendizado de máquina tarefas. Envolve diversas técnicas para melhorar a qualidade das imagens, reduzir ruído e extrair recursos significativos.



### Visualizando a imagem
Estamos utilizando as diferentes bandas (dispostas nos arquivos tifs em _./images_) agregando por duas maneiras diferentes. Testes utilizando o PCA e a principal delas, através da agregação dos pixels utilizando a média.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

class ImageMerge:
  def __init__(self, image_paths):
    self.images = []
    self.target_size = (1200, 1200)

    for path in image_paths:
      image = self.load_image(path)

      if image is not None:
        self.images.append(image)

  def load_image(self, path):
    image = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    if image is None:
      print(f"Failed to load image: {path}")
      return None

    if image.shape[:2] != self.target_size:
      image = cv2.resize(image, self.target_size[::-1])

    return image


  def merge_images(self):
    if not self.images:
      raise ValueError("Nenhuma imagem foi carregada no pipeline.")

    # Cada pixel na imagem resultante é a média dos pixels correspondentes de todas as imagens.
    merged_image = np.mean(self.images, axis=0, dtype=np.float32)
    
    # normalizando para valores de 0 a 255
    merged_image = cv2.normalize(merged_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return merged_image

  def merge_images_pca(self, n_components=3):
    if not self.images:
      raise ValueError("Nenhuma imagem foi carregada no pipeline.")

    # Empilhar todas as imagens em uma matriz 2D (pixels x bandas)
    data = np.stack([img.ravel() for img in self.images], axis=-1)

    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data)

    merged_image = principal_components.reshape(self.images[0].shape[:2] + (n_components,))
    merged_image = cv2.normalize(merged_image, None, 0, 255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)

    return merged_image

  # O método abaixo permite visualizar a imagem com a resulução de seu tamanho
  def view_image(self, image):
    dpi = 100  
    height, width = image.shape[:2]
    figsize = width / float(dpi), height / float(dpi)  

    plt.figure(figsize=figsize, dpi=dpi)  

    if image.ndim == 2 or image.shape[2] == 1:
      plt.imshow(image, cmap='gray')  

    else:
      plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    plt.axis('off')
    plt.show()

In [ ]:
image_paths = ['../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b12.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b2.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b3.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b4.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b5.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b6.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b7.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8a.tif' ]

pipe = ImageMerge(image_paths)
# int(pipe.images[1])
np.max(pipe.images)

In [ ]:
# Usando média como mescla
merged_image_mean = pipe.merge_images()
pipe.view_image(merged_image_mean)

In [ ]:
# Uma visualização rudimentar utilizando PCA
merged_image_pca = pipe.merge_images_pca()
pipe.view_image(merged_image_pca)


### Analisando imagem crua através do CLAHE (Contrast Limited Adaptive Histogram Equalization)

O método de equalização de histograma é usado em processamento de imagens para melhorar o contraste. Ele redistribui os níveis de intensidade dos pixels em uma imagem, resultando em uma distribuição de intensidades mais uniforme. Isso pode ajudar a melhorar a qualidade visual das imagens, destacando detalhes que estavam anteriormente ocultos devido a baixo contraste.

[Referência](https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html)

In [ ]:
hist, bins = np.histogram(merged_image_mean.flatten(),256,[0,256])
 
cdf = hist.cumsum()
cdf_normalized = cdf * float(hist.max()) / cdf.max()
 
plt.plot(cdf_normalized, color = 'b')
plt.hist(merged_image_mean.flatten(),256,[0,256], color = 'r')

plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show(), plt.imshow(merged_image_mean, cmap='gray')

In [ ]:
# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(merged_image_mean) 

plt.imshow(cl1, cmap='gray')

In [ ]:
plt.imshow(merged_image_mean, cmap='gray')

In [ ]:
# Calculando o histograma para a imagem "equalizada"
hist, bins = np.histogram(cl1.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max() / cdf.max()

plt.plot(cdf_normalized, color = 'b')
plt.hist(cl1.flatten(),256,[0,256], color = 'r')

plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()


## Pipeline de processamento

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class ImageProcessingPipeline:
    def __init__(self, images):
        self.images = images  

    def normalize_image(image):
        return image / 255.0

    def resize_image(image, target_size=(1200, 1200)):
        return cv2.resize(image, target_size)

    def apply_clahe(self, image):
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        return clahe.apply(image.astype(np.uint8))

    def crop_image(image, crop_size=(200, 200)):
        crops = []
        for i in range(0, image.shape[0], crop_size[0]):
            for j in range(0, image.shape[1], crop_size[1]):
                crop = image[i:i+crop_size[0], j:j+crop_size[1]]
                if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                    crops.append(crop)
        return crops

    def augment_images(self, image):
        aug_images = []
        for angle in [0, 90, 180, 270]:
            rotated = self.rotate_image(image, angle)
            aug_images.append(rotated)
            aug_images.append(cv2.flip(rotated, 1))
        return aug_images

    def rotate_image(image, angle):
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h))

    def process_images(self):
        processed_images = []
        for img in self.images:
            clahe_img = self.apply_clahe(img)
            norm_img = ImageProcessingPipeline.normalize_image(clahe_img)
            resized_img = ImageProcessingPipeline.resize_image(norm_img)
            cropped_images = self.crop_image(resized_img)
            for crop in cropped_images:
                augmented_imgs = self.augment_images(crop)
                processed_images.extend(augmented_imgs)
        return processed_images

    def show_image(self, image):
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.show()

pipeline = ImageProcessingPipeline([merged_image_mean])
processed_images = pipeline.process_images()

# Mostrar algumas das imagens processadas
for img in processed_images[:8]:  # Mostra as primeiras 8 imagens processadas
    pipeline.show_image(img)

len(processed_images)

# Testes Unitários

Os testes unitários são cruciais para garantir a qualidade do código, identificando bugs e garantindo que cada unidade funcione conforme o esperado.

## Mesclagem de imagens
Abaixo estamos testando se as funções responsáveis por trazer visibilidade a imagem estão funcionando corretamente. 

**Load Image**

In [ ]:
"""
O teste abaixo visa verificar se a função load_image está funcionando corretamente.

Como input, temos uma série de imagens .tif que pode ser encontradas na pasta images.

O resultado esperado é um array de imagens em tons de cinza com dimensões (1200, 1200).
"""

image_path = '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif'
pipe = ImageMerge([image_path])
pipe.images # A função load_image é carregada na inicialização da classe ImageMerge

# Visualizando a imagem e comparando com a imagem original 
print("Imagem GERADA:")
plt.imshow(pipe.images[0], cmap='gray')


In [ ]:
image_original = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print("Imagem ORIGINAL:")
plt.imshow(image_original, cmap='gray')

A imagem foi gerada com tamanho maior do que a original, pois, foi passado justamente o valor de 1200 pixels como target size

**Merge Images**

In [ ]:
"""
O teste abaixo visa verificar se a função merge_images está funcionando corretamente.

O input, neste caso, é uma lista de imagens em tons de cinza com dimensões (1200, 1200).

O objetivo é retornar uma única matriz, normalizada entre 0 a 255, que possui a média das imagens
"""
import numpy as np
import cv2

# Para visualizar a média iremos replicar a pipeline com mais imagens:
image_path = ['../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b12.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b2.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b3.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b4.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b5.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b6.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b7.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8a.tif' ]
pipe = ImageMerge(image_path)

# Aplicando o algoritmo desejado para mesclar as imagens
images = pipe.images
expected_merge_image = np.mean(images, axis=0, dtype=np.float32)
expected_merge_image = cv2.normalize(expected_merge_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Aplicando algoritmo da classe ImageMerge
merge_image = pipe.merge_images()

# Verificando se a imagem gerada é igual a esperada
np.testing.assert_array_equal(merge_image, expected_merge_image)


**Merge Images With PCA**

In [ ]:
"""
O teste abaixo visa verificar se a função merge_images_pca está funcionando corretamente.

O input, neste caso, é uma lista de imagens em tons de cinza com dimensões (1200, 1200).

O resultado esperado de merge_images_pca é uma representação condensada das imagens originais
que destaca suas características mais informativas ou variáveis, estruturada de forma que possa 
ser visualizada como uma imagem multi-canal.
"""

# Aplicando algoritmo da classe ImageProcessingPipeline
image_path = ['../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b12.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b2.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b3.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b4.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b5.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b6.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b7.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8a.tif' ]
pipe = ImageMerge(image_path)

merged_image_pca = pipe.merge_images_pca(3)

# Para testar, vamos apenas imprimir as dimensões e tipo para simplificar
print(merged_image_pca.shape, merged_image_pca.dtype)


In [ ]:
plt.imshow(merged_image_pca)


**View Image**

In [ ]:
"""
O teste abaixo visa verificar se a função view_image está funcionando corretamente.

O input, neste caso, é uma imagem, como uma matriz. 

O objetivo é poder visualizar a imagem, em tons de cinza, quando trata-se de uma matriz 2d ou quando o terceiro dimensionamento tem tamanho 1. E colorido, caso contrário.
"""

# Testando para uma imagem em tons de cinza
pipe.view_image(merged_image_mean)

In [ ]:
# Visualizando uma imagem colorida
pipe.view_image(merged_image_pca)

## Pipeline de processamento

Abaixo estamos testando se as funções responsáveis por realizar normalizações e manipulações na imagem estão funcionando corretamente.

**Normalize image**

In [ ]:
""" 
O teste abaixo visa verificar se a função normalize_image está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O objetivo é fazer com que os valores dos pixels da imagem estejam no intervalo [0, 1].
"""
image_before_normalization = cv2.imread('../../data/dataset_inteli_test/tci_pngs/595_2019-8-14_S2L1C_21JYJ_TCI.png', cv2.IMREAD_UNCHANGED)
print("Imagem fora do intervalor: ", image_before_normalization)


In [ ]:
image_after_normalization = ImageProcessingPipeline.normalize_image(image_before_normalization)
print("Imagem dentro do intervalo: ", image_after_normalization)

**Resize image**

In [ ]:
"""
O teste abaixo visa verificar se a função resize_image está funcionando corretamente.

O input, é uma imagem, como uma matriz. 

O output é uma imagem normalizada com dimensões (1200, 1200).
"""

image_before_resize = cv2.imread('../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif', cv2.IMREAD_UNCHANGED)
print("Tamanho da imagem, antes do processamento: ", image_before_resize.shape)

image_after_resize = ImageProcessingPipeline.resize_image(image_before_resize)
print("Tamanho da imagem, após o processamento: ", image_after_resize.shape)


**Apply Clahe**

In [ ]:
"""
O teste abaixo visa verificar se a função apply_clahe está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O output é uma imagem com contraste melhorado.
"""
image_path = ['../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b11.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b12.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b2.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b3.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b4.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b5.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b6.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b7.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8.tif', '../../data/dataset_inteli_test/images/595_2019-8-14_S2L1C_21JYJ/b8a.tif' ]
image_before_clahe = ImageMerge(image_path).merge_images()
print("Imagem antes do CLAHE: ")
plt.imshow(image_before_clahe, cmap='gray')




In [ ]:

image_processing_pipe = ImageProcessingPipeline([image_before_clahe])
image_after_clahe = image_processing_pipe.apply_clahe(image_before_clahe)
print("Imagem após o CLAHE: ")
plt.imshow(image_after_clahe, cmap='gray')


**Crop Images**

In [ ]:
"""
O teste abaixo visa verificar se a função crop_image está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O output é uma lista de 36 imagens cortadas, com dimensões (200, 200).
"""

image_before_crop = cv2.imread('../../data/dataset_inteli_test/tci_pngs/595_2019-8-14_S2L1C_21JYJ_TCI.png', cv2.IMREAD_UNCHANGED)
print("Tamanho da imagem, antes do processamento: ", image_before_crop.shape)

crops = ImageProcessingPipeline.crop_image(image_before_crop)
print("Número de imagens cortadas: ", len(crops))
print("Tamanho das imagens cortadas: ", crops[0].shape)

for crop in crops:
		plt.imshow(crop, cmap='gray')
		plt.show()

**Rotate images**

In [ ]:
"""
O teste abaixo visa verificar se a função rotate_image está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O output é uma imagem rotacionada, para este teste, em 90 graus.
"""

image_before_rotate = cv2.imread('../../data/dataset_inteli_test/tci_pngs/595_2019-8-14_S2L1C_21JYJ_TCI.png', cv2.IMREAD_UNCHANGED)
plt.imshow(image_before_rotate, cmap='gray')

In [ ]:
image_after_rotate = ImageProcessingPipeline.rotate_image(image_before_rotate, 90)
plt.imshow(image_after_rotate, cmap='gray')


**Augment images**

In [ ]:
"""
O teste abaixo visa verificar se a função augment_images está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O output é uma lista de 8 imagens, sendo 4 rotacionadas em 90, 180, 270 graus e 4 espelhadas.
"""

image_before_augment = cv2.imread('../../data/dataset_inteli_test/tci_pngs/595_2019-8-14_S2L1C_21JYJ_TCI.png', cv2.IMREAD_UNCHANGED)
plt.imshow(image_before_augment, cmap='gray')

In [ ]:
image_processing_pipe = ImageProcessingPipeline([image_before_augment])
augmented_images = image_processing_pipe.augment_images(image_before_augment)
print("Número de imagens aumentadas: ", len(augmented_images))

for img in augmented_images:
		plt.imshow(img, cmap='gray')
		plt.show()

**Process images**

In [ ]:
"""
O teste abaixo visa verificar se a função process_images está funcionando corretamente.

O input, é uma lista de imagens, como matrizes.

O output é uma lista de imagens processadas, normalizadas, redimensionadas, cortadas, aumentadas e com contraste melhorado.
"""

image_processing_pipe = ImageProcessingPipeline([merged_image_mean])
process_images = image_processing_pipe.process_images() 

# É esperado um conunto de imagens de 288: 
# 1 imagem -> Após ser recortada -> 36 imagens
# 36 imagens -> Após ser rotacionado 4x -> 144 
# 144 -> Após ser refletida horizontalmente -> 288
print("Número de imagens processadas: ", len(process_images)) 

for img in process_images[:8]:  # Mostra as primeiras 8 imagens processadas
    image_processing_pipe.show_image(img)


**Show image**

In [ ]:
"""
O teste abaixo visa verificar se a função show_image está funcionando corretamente.

O input, é uma imagem, como uma matriz.

O objetivo é poder visualizar a imagem.
"""

image = cv2.imread('../../data/dataset_inteli_test/tci_pngs/595_2019-8-14_S2L1C_21JYJ_TCI.png', cv2.IMREAD_UNCHANGED)
image_processing_pipe = ImageProcessingPipeline([image])
image_processing_pipe.show_image(image)
